In [1]:
# Loading Libraries
import pandas as pd
import sklearn as sk
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

import log_hyperu as hyperu
import tgr as tgr

import time

In [2]:
# Generating the data sets

torch.manual_seed(12345)

# Structure of the data sets
## Set X_A: Sparse Coefficients with many data points
## Set X_B: Dense Coefficients with many data points
## Set X_C: Sparse Coefficients with few data points
## Set X_D: Dense Coefficients with few data points


### Set X_A
variables = 12
sample = 100
true_coefs = torch.tensor([[0],[0],[5.3],[4.2],[0],[0],[6.9],[0],[0],[0],[0],[0]])
X = torch.randn(sample, variables)
Y = X @ true_coefs + torch.randn(sample, 1) * 0.1
print(X)

tensor([[-1.4798,  0.4873, -3.0128,  ...,  1.9991,  0.7848, -1.0195],
        [-0.2106,  0.6268,  0.9318,  ..., -0.5429,  0.4307, -1.9257],
        [ 1.2756, -1.1316,  0.8680,  ..., -0.8407, -0.3963, -0.2591],
        ...,
        [-1.3737,  0.0184,  1.1507,  ..., -0.8902,  0.5128, -0.5483],
        [-0.4525,  1.2544,  1.0292,  ...,  0.3971, -0.7133, -0.6877],
        [-0.3455, -0.0047, -0.2237,  ...,  0.0928,  0.3904,  1.0149]])


In [21]:
iterations = 4000
starttime = time.time()
trained_model2, coefs, loss_of_optimization = tgr.TripleGammaModel(X, Y, 1, 1, 30, 1, True, num_epochs=iterations) # Covariates, Targets, Penalty, a, c, kappa, normalization=True
endtime = time.time()
print(f'With {iterations} iterations, TG Regularization took {round(endtime-starttime,4)} seconds. On this system, that is roughly {(endtime-starttime)/iterations} seconds per iteration.')
coefficients = trained_model2.linear.weight.detach().numpy()
#intercept = trained_model2.linear.bias.item()

# For LASSO Imitation: (X, Y, 1, 1, 30, 1, True, num_epochs=2000) # Covariates, Targets, Penalty, a, c, kappa, normalization=True

#print("Coefficients General Function:", coefficients)
#print("Intercept General Function:", intercept)
print(coefficients[0].tolist())

Epoch [100/4000], Loss: 55.32939529418945
Epoch [200/4000], Loss: 45.14422607421875
Epoch [300/4000], Loss: 36.81026840209961
Epoch [400/4000], Loss: 30.06134033203125
Epoch [500/4000], Loss: 24.816471099853516
Epoch [600/4000], Loss: 20.907426834106445
Epoch [700/4000], Loss: 18.205690383911133
Epoch [800/4000], Loss: 16.529497146606445
Epoch [900/4000], Loss: 15.707762718200684
Epoch [1000/4000], Loss: 15.401206016540527
Epoch [1100/4000], Loss: 15.280069351196289
Epoch [1200/4000], Loss: 15.237548828125
Epoch [1300/4000], Loss: 15.218250274658203
Epoch [1400/4000], Loss: 15.218034744262695
Epoch [1500/4000], Loss: 15.204828262329102
Epoch [1600/4000], Loss: 15.213384628295898
Epoch [1700/4000], Loss: 15.21099853515625
Epoch [1800/4000], Loss: 15.21127700805664
Epoch [1900/4000], Loss: 15.211557388305664
Epoch [2000/4000], Loss: 15.209331512451172
Epoch [2100/4000], Loss: 15.209222793579102
Epoch [2200/4000], Loss: 15.209370613098145
Epoch [2300/4000], Loss: 15.206412315368652
Epoch 

In [24]:
#LASSO
# Define your model
starttime = time.time()
class LinearRegressionLasso(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearRegressionLasso, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.linear(x)

# Define hyperparameters
input_size = 12
output_size = 1
learning_rate = 0.01
num_epochs = 4000
lambda_lasso = 1  # L1 regularization parameter

# Create the model
model = LinearRegressionLasso(input_size, output_size)

# Define loss function (MSE loss with L1 regularization)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X)
    loss = criterion(outputs, Y)
    
    # L1 regularization term
    l1_reg = torch.tensor(0.)
    for param in model.parameters():
        l1_reg += torch.norm(param, p=1)
    
    # Total loss with L1 regularization
    loss += lambda_lasso * l1_reg
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

endtime = time.time()
print(f'With {num_epochs} iterations, Regular LASSO Regularization took {round(endtime-starttime,4)} seconds. On this system, that is roughly {(endtime-starttime)/num_epochs} seconds per iteration.')

# After training, you can access the learned coefficients
for name, param in model.named_parameters():
    if 'weight' in name:
        print(f'{name}: {param.data.tolist()}')


Epoch [100/4000], Loss: 19.0086
Epoch [200/4000], Loss: 15.6859
Epoch [300/4000], Loss: 15.4875
Epoch [400/4000], Loss: 15.4785
Epoch [500/4000], Loss: 15.4944
Epoch [600/4000], Loss: 15.4819
Epoch [700/4000], Loss: 15.4776
Epoch [800/4000], Loss: 15.4981
Epoch [900/4000], Loss: 15.4745
Epoch [1000/4000], Loss: 15.4835
Epoch [1100/4000], Loss: 15.4936
Epoch [1200/4000], Loss: 15.4895
Epoch [1300/4000], Loss: 15.4747
Epoch [1400/4000], Loss: 15.4798
Epoch [1500/4000], Loss: 15.4760
Epoch [1600/4000], Loss: 15.4725
Epoch [1700/4000], Loss: 15.4822
Epoch [1800/4000], Loss: 15.4826
Epoch [1900/4000], Loss: 15.4856
Epoch [2000/4000], Loss: 15.4977
Epoch [2100/4000], Loss: 15.4850
Epoch [2200/4000], Loss: 15.4764
Epoch [2300/4000], Loss: 15.4907
Epoch [2400/4000], Loss: 15.4766
Epoch [2500/4000], Loss: 15.4710
Epoch [2600/4000], Loss: 15.4752
Epoch [2700/4000], Loss: 15.4836
Epoch [2800/4000], Loss: 15.4781
Epoch [2900/4000], Loss: 15.4752
Epoch [3000/4000], Loss: 15.4802
Epoch [3100/4000], 

In [25]:
# Normale Regression

starttime = time.time()
class LinearRegressionLasso(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearRegressionLasso, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.linear(x)

# Define hyperparameters
input_size = 12
output_size = 1
learning_rate = 0.01
num_epochs = 4000
lambda_lasso = 0 # To get OLS Estimate

# Create the model
model = LinearRegressionLasso(input_size, output_size)

# Define loss function (MSE loss with L1 regularization)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X)
    loss = criterion(outputs, Y)
    
    # L1 regularization term
    l1_reg = torch.tensor(0.)
    for param in model.parameters():
        l1_reg += torch.norm(param, p=1)
    
    # Total loss with L1 regularization
    loss += lambda_lasso * l1_reg
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

endtime = time.time()
print(f'With {num_epochs} iterations, Regular OLS took {round(endtime-starttime,4)} seconds. On this system, that is roughly {(endtime-starttime)/num_epochs} seconds per iteration.')

# After training, you can access the learned coefficients
for name, param in model.named_parameters():
    if 'weight' in name:
        print(f'{name}: {param.data.tolist()}')


Epoch [100/4000], Loss: 4.2589
Epoch [200/4000], Loss: 0.4515
Epoch [300/4000], Loss: 0.0659
Epoch [400/4000], Loss: 0.0175
Epoch [500/4000], Loss: 0.0108
Epoch [600/4000], Loss: 0.0098
Epoch [700/4000], Loss: 0.0096
Epoch [800/4000], Loss: 0.0096
Epoch [900/4000], Loss: 0.0096
Epoch [1000/4000], Loss: 0.0096
Epoch [1100/4000], Loss: 0.0096
Epoch [1200/4000], Loss: 0.0096
Epoch [1300/4000], Loss: 0.0096
Epoch [1400/4000], Loss: 0.0096
Epoch [1500/4000], Loss: 0.0096
Epoch [1600/4000], Loss: 0.0096
Epoch [1700/4000], Loss: 0.0096
Epoch [1800/4000], Loss: 0.0096
Epoch [1900/4000], Loss: 0.0096
Epoch [2000/4000], Loss: 0.0096
Epoch [2100/4000], Loss: 0.0096
Epoch [2200/4000], Loss: 0.0096
Epoch [2300/4000], Loss: 0.0096
Epoch [2400/4000], Loss: 0.0096
Epoch [2500/4000], Loss: 0.0096
Epoch [2600/4000], Loss: 0.0096
Epoch [2700/4000], Loss: 0.0096
Epoch [2800/4000], Loss: 0.0096
Epoch [2900/4000], Loss: 0.0096
Epoch [3000/4000], Loss: 0.0096
Epoch [3100/4000], Loss: 0.0096
Epoch [3200/4000]